You can use this notebook to develop your answers. Make sure to look at intermediate results using `take()` for debugging.

In [21]:
import json 

## Load data into RDDs
usersRDD = sc.textFile("datafiles/se_users.json").map(json.loads)
postsRDD = sc.textFile("datafiles/se_posts.json").map(json.loads)
playRDD = sc.textFile("datafiles/play.txt")
logsRDD = sc.textFile("datafiles/NASA_logs_sample.txt")
amazonInputRDD = sc.textFile("datafiles/amazon-ratings.txt")
nobelRDD = sc.textFile("datafiles/prize.json").map(json.loads)
amazonBipartiteRDD = amazonInputRDD.map(lambda x: x.split(" ")).map(lambda x: (x[0], x[1])).distinct()

In [65]:
for t in postsRDD.take(3): print(t.get("tags"))

<mysql><version-control><schema>
<database-design><erd>
None


Task 1 (0.25): Use filter to find all posts where tags are not null (None in python) and that are tagged 'postgresql-9.4', and then a map so that the output RDD has tuples of the form: (ID, Title, Tags). Note that postsRDD contains dictionaries -- see the contents by running postsRDD.take(10).

In [66]:
def task1(postsRDD):
    res = postsRDD.filter(
        lambda x: x.get("tags")!= None).filter(
        lambda x: "postgresql-9.4" in x.get("tags")).map(
        lambda x: (x.get("id"), x.get("title"), x.get("tags"))
    )
    return res 
a = task1(postsRDD)

<class 'pyspark.rdd.PipelinedRDD'>


Task 2 (0.25): Use flatMap on the postsRDD to create an RDD (ID, Tag), listing all the tags for each post as a separate tuple. If a post has no tags, it should not appear in the output RDD.

In [74]:
def task2FlatMapper(dic):
    res = []
    tags = dic.get("tags").replace("<", "").replace(">", " ").split(" ")
    tags.pop()
    for i in tags:
        res.append( (dic.get("id"), i)
        )
    return res
    

def task2(postsRDD):
    return postsRDD.filter(
        lambda x: x.get("tags")!= None).flatMap(
        task2FlatMapper
    )
a = task2(postsRDD)

In [ ]:
def task2FlatMapper(dic):
    res = []
    tags = dic.get("tags").replace("<", "").replace(">", " ").split(" ")
    tags.pop()
    for i in tags:
        res.append( (dic.get("id"), i)
        )
    return res
    

def task2(postsRDD):
    return postsRDD.filter(
        lambda x: x.get("tags")!= None).flatMap(
        task2FlatMapper
    )
a = task2(postsRDD)

In [76]:
for t in a.take(10): print(t)

(2, 'mysql')
(2, 'version-control')
(2, 'schema')
(3, 'database-design')
(3, 'erd')
(5, 'nosql')
(5, 'rdbms')
(5, 'database-recommendation')
(6, 'postgresql')
(6, 'replication')


In [61]:
type(postsRDD)

pyspark.rdd.PipelinedRDD

In [36]:
dij = {"name":"joshua"}

In [38]:
"os" in dij.get("name")

True

In [69]:
tagss = "<tag1><tag2>"
print(tagss.replace("<", "").replace(">", " ").split(" "))

['tag1', 'tag2', '']


In [71]:
ll = [1,2,3]
ll.pop()
ll

[1, 2]